## 匯入模組


In [ ]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
!pip install openai==0.28

print("Completed!")

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset
import openai
import pandas
import tempfile
import os
import subprocess  # 用來調用 ffmpeg 進行音頻轉換
from flask import Flask, request, jsonify
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage, AudioMessage
import speech_recognition as sr
import dateparser
from datetime import datetime
import openpyxl
from openpyxl import Workbook
import schedule
import time
import uuid
import threading
from flask_ngrok import run_with_ngrok
  

##  LINE Messaging API 設定

In [ ]:
line_bot_api = LineBotApi('Line Token')
handler = WebhookHandler('Line Secret')

## OpenAI key 設定

In [ ]:
# OpenAI API Key
openai.api_key = 'OpenAPIKey'

# 初始化 Flask 和 LINE Bot

In [ ]:
app = Flask(__name__) #app name

## 接收 LINE 的 Webhook 事件

In [ ]:
@app.route("/", methods=['POST'])
def callback():
    # 確認 LINE 的簽名是否正確
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        return jsonify({'message': 'Invalid signature'}), 400
    return 'OK',200

# 解析時間的函數

In [ ]:
def parse_time(user_input_time):
    parsed_time = dateparser.parse(user_input_time)
    if parsed_time:
        return parsed_time.strftime('%Y-%m-%d %H:%M:%S')
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

## 處理收到的文字訊息

In [ ]:
user_states = {}
@handler.add(MessageEvent, message=TextMessage)
def handle_text_message(event):
    text = event.message.text
    user_id = event.source.user_id
    if text.startswith("會議回顧"):
        keyword = text.replace("會議回顧", "").strip()
        response_message = search_summary(keyword)
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=response_message))
    if user_states.get(user_id) == "Reminders":
        formatted_time = parse_time("現在")
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一個有用的提醒功能機器人，幫助用戶記錄需提醒的事情"},
                {"role": "user", "content": f"請把以下提醒的事項整理成Time和Content，時間格式為 YYYY-MM-DD HH:MM，並以Database語法存入，只需要寫語法就好，不需要額外補充而且時間一定要準確：現在時間是：{formatted_time},内容是{text}"}
            ]
        )
        
        response_text = response['choices'][0]['message']['content']
        #line_bot_api.reply_message(event.reply_token, TextSendMessage(text=response_text))
        # 保存提醒事項和轉換文本到 Excel
        response_message = handle_transcript_and_reminder(user_id, response_text, text)
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=response_message))
        user_states[user_id] = ""

    elif text == "請輸入會議音檔" and user_states.get(user_id) not in ["Recording", "Reminders","Summary_check"]:
        user_states[user_id] = "Recording"

    elif text == "請輸入需提醒事項" and user_states.get(user_id) not in ["Recording", "Reminders","Summary_check"]:
        user_states[user_id] = "Reminders"
    elif text == "會議回顧" and user_states.get(user_id) not in ["Recording", "Reminders","Summary_check"]:
        print('會議回顧中')
        user_states[user_id] = "Summary_check"

def summarize_with_openai(transcript):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "你是一個有用的會議記錄助手，幫助用戶開會的重點進行記錄，列點給我，不要多說別的東西。"},
            {"role": "user", "content": f"請總結以下文本的重點：{transcript}"}
        ]
    )
    
    return response['choices'][0]['message']['content']



## 處理音檔

In [ ]:
import os
import subprocess
import uuid
from concurrent.futures import ThreadPoolExecutor
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

# 音訊訊息處理函數
@handler.add(MessageEvent, message=AudioMessage)
def handle_audio_message(event):
    user_id = event.source.user_id
    message_id = event.message.id
    message_content = line_bot_api.get_message_content(message_id)

    # 儲存音頻到本地
    current_directory = os.getcwd()
    audio_path = os.path.join(current_directory, f'audio_file_{uuid.uuid4()}.m4a')

    # 將音檔內容寫入指定位置
    with open(audio_path, 'wb') as f:
        f.write(message_content.content)

    try:
        # 將 .m4a 文件轉換為 .wav 文件
        wav_path = convert_to_wav(audio_path)
        # 使用本地語音轉文字方法將語音轉為文本
        transcript = changetotext(wav_path)

        # 根據用戶狀態進行相應處理
        if user_states.get(user_id) == "Recording":
            summary = summarize_with_openai(transcript)
            print('已成功存到資料庫')
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=summary))
        elif user_states.get(user_id) == "Reminders":
            formatted_time = parse_time("現在")
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "你是一個有用的提醒功能機器人，幫助用戶記錄需提醒的事情"},
                    {"role": "user", "content": f"請把以下提醒的事項整理成Time和Content，時間格式為 YYYY-MM-DD HH:MM，並以Database語法存入，只需要寫語法就好，不需要額外補充而且時間一定要準確：現在時間是：{formatted_time},内容是{transcript}"}
                ]
            )
            response_text = response['choices'][0]['message']['content']
            response_message = handle_transcript_and_reminder(user_id, response_text)
            line_bot_api.reply_message(event.reply_token, TextSendMessage(text=response_message))
        user_states[user_id] = ""
        
    except Exception as e:
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=f"抱歉，無法處理您的錄音。錯誤信息: {str(e)}"))

# 將 .m4a 轉換為 .wav 的函數
def convert_to_wav(audio_path):
    wav_path = audio_path.replace('.m4a', '.wav')
    ffmpeg_path = r"C:\ffmpeg\bin\ffmpeg.exe"
    try:
        print("開始轉換音頻...")
        result = subprocess.run(
            [ffmpeg_path, "-i", audio_path, wav_path],
            capture_output=True, text=True
        )
        
        if result.returncode != 0:
            raise Exception(f"音頻轉換失敗: {result.stderr}")
        print("轉換成功")
        return wav_path
    except subprocess.CalledProcessError as e:
        raise Exception(f"音頻轉換失敗: {e}")

# 使用 Whisper 模型進行轉錄
def changetotext(wav_path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3-turbo"

    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device
    )

    # 分割音訊檔案
    def split_audio(wav_path, segment_length=25):
        output_dir = "audio_segments"
        os.makedirs(output_dir, exist_ok=True)
        subprocess.run([
            "ffmpeg", "-i", wav_path,
            "-f", "segment",
            "-segment_time", str(segment_length),
            os.path.join(output_dir, "segment_%03d.wav")
        ])
        return [os.path.join(output_dir, f) for f in sorted(os.listdir(output_dir)) if f.endswith(".wav")]

    # 轉錄單個音訊片段
    def transcribe_segment(segment_path):
        result = pipe(segment_path, generate_kwargs={"language": "Chinese"})
        return result["text"] if "text" in result else ""
    
    # 並行處理所有片段並合併結果
    def transcribe_all_segments(segments, chunk_size=4):
        with ThreadPoolExecutor(max_workers=chunk_size) as executor:
            results = list(executor.map(transcribe_segment, segments))
        return " ".join(results)

    # 進行音訊分割並轉錄
    segments = split_audio(wav_path)
    transcript = transcribe_all_segments(segments, chunk_size=4)

    print("Transcript: " + transcript)
    return transcript

# 定期檢查提醒

In [ ]:
def check_reminders():
    excel_file_path = 'reminders_database.xlsx'
    workbook = openpyxl.load_workbook(excel_file_path)
    sheet = workbook.active

    # 獲取當前時間
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M')

     # 遍歷表格，檢查時間是否到達
    for row in sheet.iter_rows(min_row=2, values_only=True):  # 從第二行開始，忽略標題行
        user_id, reminder_time, reminder_content = row[:3]  # 根據新順序提取 user_id、時間和內容
        if reminder_time == current_time:
            # 發送提醒消息，確保 user_id 有效
            if user_id:
                line_bot_api.push_message(user_id, TextSendMessage(text=f"提醒時間到！內容是: {reminder_content}"))
            else:
                print(f"無效的 user_id，無法發送提醒給: {reminder_content}")

# 開始一個定期執行的任務來檢查提醒
def start_scheduler():
    schedule.every(1)


# 啟動 Flask 應用

In [ ]:
if __name__ == "__main__":
    app.run(port=5000, debug=True, use_reloader=False)
